In [ ]:
import ray
import torch
from rllib_setup import get_env_continuous


In [ ]:
env = get_env_continuous()
env.action_space


In [ ]:
ray.init()


In [ ]:
env = get_env()


In [ ]:
env_name = "VJS"


In [ ]:
ray.rllib.utils.check_env(env)


In [ ]:
from ray.tune import register_env
register_env(env_name,lambda config: get_env())


In [ ]:
from ray.rllib.policy.policy import PolicySpec
test_env = get_env()
obs_space = test_env.observation_space
act_space = test_env.action_space
def policies(agent_ids):
    return {
        str(i): PolicySpec(
            # observation_space=obs_space,
            # action_space=act_space,
            config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }


In [ ]:
action = test_env.action_space_sample()
test_env.step(action)


In [ ]:
test_env.action_space


In [ ]:
from ray.tune.registry import get_trainable_cls
import os
config = (
    get_trainable_cls("MADDPG")
    .get_default_config()
    .environment(env=env_name)
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
)
(
    config.framework("tf")
    # .environment(env_config={"actions_are_logits": True})
    .training(num_steps_sampled_before_learning_starts=100)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: str(
            agent_id
        ),
    )
)


In [ ]:
%load_ext rich


In [ ]:
from ray import air, tune
stop = {
    "training_iteration": 10000,
}
results = tune.Tuner(
    "MADDPG",
    run_config=air.RunConfig(stop=stop, verbose=2),
    param_space=config,
).fit()


In [ ]:
import ray
from ray.rllib.algorithms.ddpg.ddpg import DDPGConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env
import rich  
import rllib_setup

env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): PolicySpec(
            observation_space=obs_space,
            action_space=act_space,
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }




In [ ]:
config = (
    DDPGConfig()
    .training(lr=0.01)
    .resources(num_gpus=1)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
)
# config.batch_mode = "complete_episodes"
rich.print(config.to_dict())


In [ ]:
algo = config.build(env=env_name)


In [ ]:
algo.train()


In [ ]:
from ray import tune
from ray import air


In [ ]:
tune.Tuner(  
    "DDPG",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit()


In [ ]:
from ray.rllib.algorithms.sac import SACConfig
from ray.tune import register_env
import rllib_setup

env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }
    
    
config = SACConfig().training(gamma=0.9, lr=0.01)
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=4).multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
config.batch_mode = "complete_episodes"
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env=env_name)  
algo.train()


In [ ]:
from ray.rllib.algorithms.maddpg.maddpg import MADDPGConfig
config = MADDPGConfig()
print(config.replay_buffer_config)  
replay_config = config.replay_buffer_config.update(  
    {
        "capacity": 100000,
        "prioritized_replay_alpha": 0.8,
        "prioritized_replay_beta": 0.45,
        "prioritized_replay_eps": 2e-6,
    }
)
config.training(replay_buffer_config=replay_config)   
config = config.resources(num_gpus=0)   
config = config.rollouts(num_rollout_workers=4)   
config = config.environment(env=env_name)  
algo = config.build()  
algo.train()  


In [1]:
import ray
from ray.rllib.algorithms.ddpg.ddpg import DDPGConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env

import rllib_setup

ray.init()
env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(60),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }


config = (
    DDPGConfig().rollouts(num_rollout_workers=20)
    .training(lr=0.01)
    .resources(num_gpus=1)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
    .environment(disable_env_checking=True)
)
config.batch_mode = "complete_episodes"
print(config.to_dict())
# Build a Algorithm object from the config and run one training iteration.
algo = config.build(env=env_name)
algo.train()
algo.evaluate()


2023-09-26 13:47:45,193	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:442: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/yuan/ResMan/man/lib/python3.9/site-

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 1, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': True, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_envs_pe

2023-09-26 13:47:57,703	INFO trainable.py:173 -- Trainable.setup took 11.189 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(RolloutWorker pid=2925449) 2023-09-26 13:48:41,962	WARNING env_runner_v2.py:154 -- More than 22256 observations in 22256 env steps for episode 692983022856277297 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.
2023-09-26 13:48:42,895	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing

ValueError: Cannot evaluate w/o an evaluation worker set in the Trainer or w/o an env on the local worker!
Try one of the following:
1) Set `evaluation_interval` >= 0 to force creating a separate evaluation worker set.
2) Set `create_env_on_driver=True` to force the local (non-eval) worker to have an environment to evaluate on.

(raylet) [2023-09-26 16:28:45,617 E 2922067 2922067] (raylet) node_manager.cc:3069: 7 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 1f6ea0053076213805ba22f91ea908c4d3938e814201bed0f037e38d, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-09-26 16:29:45,619 E 2922067 2922067] (raylet) node_manager.cc:3069: 7 Workers (tasks / actors) kill

In [ ]:
for i in range(10):
    print(algo.training_step()['Machine_0']['learner_stats']['actor_loss'])


In [ ]:
algo.evaluate()
